In [ ]:


def align_xl(xl1, xl2, xl3):
    cols =['interval','timestamp','x', 'y', 'z']
    xl1_df = pd.DataFrame(xl1, columns=cols)
    xl2_df = pd.DataFrame(xl2, columns=cols)
    xl3_df = pd.DataFrame(xl3, columns=cols)

    xl1_group = xl1_df.groupby('interval')
    xl2_group = xl2_df.groupby('interval')
    xl3_group = xl3_df.groupby('interval')

    xl1_w = []
    xl2_w = []
    xl3_w = []

    xl1_aligned = []
    xl2_aligned = []
    xl3_aligned = []

    xl_combined = []

    for (name1, xl1_grp), (name2, xl2_grp), (name3, xl3_grp) in zip(xl1_group, xl2_group, xl3_group):

        xl_1 = preprocess_sensor(xl1_grp.drop(columns=['interval']).values)
        xl_2 = preprocess_sensor(xl2_grp.drop(columns=['interval']).values)
        xl_3 = preprocess_sensor(xl3_grp.drop(columns=['interval']).values)

        xl_1 = xl_1[:, 1:]
        xl_2 = xl_2[:, 1:]
        xl_3 = xl_3[:, 1:]

        xl1_clipped = clip_xl(xl_1, 2)
        xl2_clipped = clip_xl(xl_2, 8)
        xl3_clipped = clip_xl(xl_3, 4)

        xl1_w.append(xl1_clipped)
        xl2_w.append(xl2_clipped)
        xl3_w.append(xl3_clipped)


        if len(xl1_w) > 3: 
            xl1_w.pop(0)
            xl2_w.pop(0)
            xl3_w.pop(0)
        
        if len(xl1_w) != 3:
            continue

        xl1_ta = time_align(xl1_w, 1000, 1000, 'xl', False)
        xl2_ta = time_align(xl2_w, 1000, 1000, 'xl', False)
        xl3_ta = time_align(xl3_w, 1000, 1000, 'xl', False)

        xl1_aligned.append(xl1_ta)
        xl2_aligned.append(xl2_ta)
        xl3_aligned.append(xl3_ta)

        final_xl = combine_xl(xl1_ta, xl2_ta, xl3_ta)
        xl_combined.append(final_xl)

    xl1_aligned = np.array(xl1_aligned).reshape(-1, 3)
    xl2_aligned = np.array(xl2_aligned).reshape(-1, 3)
    xl3_aligned = np.array(xl3_aligned).reshape(-1, 3)
    xl_combined = np.array(xl_combined).reshape(-1, 3)

    return xl1_aligned, xl2_aligned, xl3_aligned, xl_combined

def combine_xl(xl1, xl2, xl3): 

    xl1_xl3 = np.where(xl3 > 2, xl3, xl1)
    xl1_xl3 = np.where(xl3 < -2, xl3, xl1_xl3)

    final_xl = np.where(xl2 > 4, xl2, xl1_xl3)
    final_xl = np.where(xl2 < -4, xl2, final_xl)

    return final_xl

def preprocess_sensor(dat):
    """ 
    Sorts sensor data by timestamp and removes duplicates
    Params: 
        dat: sensor data 
    """
    dat = np.array(dat).astype(float)
    sorted_dat = dat[dat[:,0].argsort()]

    _, unique_indices = np.unique(sorted_dat[:,0], return_index = True)
    processed_data = sorted_dat[unique_indices]
    return processed_data


def clip_xl(xl_dat, clip):
    """
    XL sensors above certain thresholds are defective
    """
    xl_clipped = np.where(xl_dat > clip, clip, xl_dat)
    xl_clipped = np.where(xl_clipped < -1*clip, -1*clip, xl_clipped)
    return xl_clipped

def time_align(xyz, sensor_odr, interval_odr, name, interpolate=False):
    """ 
    Upsample data 2x interval odr(fourier signal), decimate data back down(lowpass filter), interpolate to desired odr, select current interval points
    """
    xyz_w = []
    len_w = []
    for i, ts in enumerate(xyz):
        xyz_w.append(ts)
        len_w.append(len(ts))

    xyz_w = np.vstack(xyz_w)

    xyz_r = resample(xyz_w, num=(sensor_odr*3)*2, axis=0)
    factor = xyz_r.shape[0]//((sensor_odr*3))
    xyz_d = decimate(xyz_r, q=factor, axis=0)
    # print('decimate', xyz_d.shape[0])

    if interpolate:
        new_points = interpolate_points(xyz_d, np.arange(xyz_d.shape[0]), interval_odr*3)
    else:
        new_points = xyz_d

    current_points = new_points[interval_odr:interval_odr*2]
    
    # print('final', np.array(current_points).shape)
    return np.array(current_points)
